In [ ]:
!pip install torchtext==0.6

ERROR: Could not find a version that satisfies the requirement Zipfile (from versions: none)
ERROR: No matching distribution found for Zipfile


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchtext
from torchtext.vocab import FastText
from torchtext.vocab import GloVe
from torchtext.data import Field
from torchtext.data import Dataset, Example
from torchtext.data import BucketIterator
import numpy as np
import pandas as pd
import os
import re
import matplotlib.pyplot as plt
import json

In [ ]:
torchtext.__version__

'0.6.0'

In [ ]:
word_to_ix = {"hello": 0, "world": 1}
embeds = nn.Embedding(2, 5)  # 2 words in vocab, 5 dimensional embeddings
lookup_tensor = torch.tensor([word_to_ix["hello"]], dtype=torch.long)
hello_embed = embeds(lookup_tensor)
print(hello_embed)

tensor([[-0.8016, -0.8072, -0.1534,  0.7872, -0.0733]],
       grad_fn=<EmbeddingBackward>)


In [ ]:
!ls '/content/drive/My Drive/490A Project/'

 Biden	'Final Dataset'   Trump


In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

biden_data_dir = '/content/drive/My Drive/490A Project/Biden'
trump_data_dir = '/content/drive/My Drive/490A Project/Trump'
biden_folders = os.listdir(biden_data_dir)
trump_folders = os.listdir(trump_data_dir)
classes = ["BIDEN", "TRUMP"]
print("Selected Biden Hashtags:", biden_folders)
print("Selected Trump Hashtags:",trump_folders)

Mounted at /content/drive
Selected Biden Hashtags: ['Icon\r', 'BidenHarris', 'Biden2020']
Selected Trump Hashtags: ['Icon\r', 'TrumpPence', 'Trump2020']


In [ ]:
import zipfile

final_data = '/content/drive/My Drive/490A Project/Final Dataset/final_dataset_no_error.zip'
biden_final_dir = "merged_no_dups_no_error/biden"
trump_final_dir = "merged_no_dups_no_error/trump"

with zipfile.ZipFile(final_data, 'r') as zipObj:
   # Extract all the contents of zip file in current directory
   zipObj.extractall()

In [ ]:
!ls "merged_no_dups_no_error/"

biden  trump


In [ ]:
# Only Preprocessing needed is removing hashtags, removing keywords, remove users and replacing links
# Lemmatising and Stemming is not necessary since word embeddings are being used
# The tokenising is done by TorchText at the time of creating the field
# Converting to lowercase is done by TorchText at the time of creating the field

URL_replacement = "[link]"

'''
def preprocess(samples):

  for index, sample in samples.iterrows():
    temp = sample["full_text"]

    temp = re.sub('#[^a-zA-Z0-9]+', '', temp)
    temp = re.sub('@[^a-zA-Z0-9]+', '', temp)
 
    temp = re.sub("http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+",
                                 URL_replacement, 
                                 temp,
                                 flags=re.MULTILINE, 
                                 )
    
    #samples["full_text"] = samples["full_text"].replace(sample["full_text"],temp)
  return temp
'''

def preprocess(text):
  temp = text
  temp = re.sub('#[^a-zA-Z0-9]+', '', temp)
  temp = re.sub('@[^a-zA-Z0-9]+', '', temp)
  temp = re.sub("http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+",
                                 URL_replacement, 
                                 temp,
                                 flags=re.MULTILINE, 
                                 )
  text = temp
  return temp

In [ ]:
TotalData = { classes[0]: [],
              classes[1]: []    
}

df = pd.DataFrame()

'''
#Add all data to Biden dataset
for dir in os.listdir(biden_data_dir):
  if dir == 'Icon\r':
    continue
  for fname in os.listdir(biden_data_dir + "/" + dir):
    if fname == "Icon\r":
      continue
    value = pd.read_json(biden_data_dir + "/" + dir + '/' + fname)
    if "full_text" not in value:
      continue
    value = preprocess(value)
    value_text = list(value["full_text"])
    value_labels = [classes[0] for i in range(len(value_text)) ]
    temp = { "Text": value_text,
             "Class": value_labels
    }
    df2 = pd.DataFrame(temp)
    df = df.append(df2, ignore_index=True)

print("Done Collecting Biden Data")


#Add all data to Trump dataset
for dir in os.listdir(trump_data_dir):
  if dir == 'Icon\r':
    continue
  for fname in os.listdir(trump_data_dir + "/" + dir):
    if fname == "Icon\r":
      continue
    value = pd.read_json(trump_data_dir + "/" + dir + '/' + fname)
    if "full_text" not in value:
      continue
    value_text = list(value["full_text"])
    value_labels = [classes[1] for i in range(len(value_text)) ]
    temp = { "Text": value_text,
             "Class": value_labels
    }
    df2 = pd.DataFrame(temp)
    df = df.append(df2, ignore_index=True)
print("Done Collecting Trump Data")
'''

for fname in os.listdir(biden_final_dir + "/"):
    if fname == "Icon\r":
      continue
    value = data = json.load(open(biden_final_dir + "/" + fname, encoding='utf-8'))
    if "full_text" not in value:
      continue
    
    value_text = preprocess(value["full_text"])
    value_labels = [classes[0]]
    temp = { "Text": value_text,
             "Class": value_labels
    }
    df2 = pd.DataFrame(temp)
    df = df.append(df2, ignore_index=True)

print("Done Collecting Biden Data")

for fname in os.listdir(trump_final_dir + "/"):
    if fname == "Icon\r":
      continue
    value = data = json.load(open(trump_final_dir + "/" + fname, encoding='utf-8'))
    if "full_text" not in value:
      continue
    value_text = value["full_text"]
    value_text = preprocess(value_text)
    value_labels = [classes[1]]
    temp = { "Text": value_text,
             "Class": value_labels
    }
    df2 = pd.DataFrame(temp)
    df = df.append(df2, ignore_index=True)

print("Done Collecting Trump Data")

print(df)

Done Collecting Biden Data
Done Collecting Trump Data
                                                     Text  Class
0       @BillStepien Bill, you've done your best on a ...  BIDEN
1       Bring it home, America. Let's prove that we be...  BIDEN
2       RT @meg_Y12: From an American friend receiving...  BIDEN
3       @Timodc Yes, Collin and Denton County both.  B...  BIDEN
4       @redshoe9 @JudithWick1 @BettyBowers @DJJudd @y...  BIDEN
...                                                   ...    ...
337301  Time to get the #RedWave STARRTTEEDD!!!!\n#Tru...  TRUMP
337302  Y’all have anymore 😳 ? Please post below ! We ...  TRUMP
337303  RT @charlybcrawford: Making Halloween Rodeo gr...  TRUMP
337304  🤮🤢 #MediaReform is a must if we're going to #M...  TRUMP
337305  Remember to #VOTE for #TrumpPence during this ...  TRUMP

[337306 rows x 2 columns]


In [ ]:
text_field = Field(
    sequential=True,
    tokenize='toktok', 
    fix_length=5,
    lower=True
)
label_field = Field(sequential=False, use_vocab=False)
# sadly have to apply preprocess manually
preprocessed_text = df['Text'].apply(
    lambda x: text_field.preprocess(x)
)
# load fastext simple embedding with 300d
text_field.build_vocab(
    preprocessed_text, 
    vectors='fasttext.simple.300d'
)
# get the vocab instance
vocab = text_field.vocab


.vector_cache/wiki.simple.vec: 293MB [00:12, 23.6MB/s]                           
100%|█████████▉| 110691/111051 [00:11<00:00, 8707.94it/s]

In [ ]:
embedding = FastText('simple')

In [ ]:
# known token, in my case print 12
print(vocab['are'])
# unknown token, will print 0
print(vocab['crazy'])

31
1232


In [ ]:
ltoi = {l: i for i, l in enumerate(df['Class'].unique())}
df['Class'] = df['Class'].apply(lambda y: ltoi[y])
class DataFrameDataset(Dataset):
    def __init__(self, df: pd.DataFrame, fields: list):
        super(DataFrameDataset, self).__init__(
            [
                Example.fromlist(list(r), fields) 
                for i, r in df.iterrows()
            ], 
            fields
        )

In [ ]:
train_dataset, test_dataset = DataFrameDataset(
    df=df, 
    fields=(
        ('text', text_field),
        ('label', label_field)
    )
).split()

100%|█████████▉| 110691/111051 [00:30<00:00, 8707.94it/s]

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(torch.cuda.current_device())

train_iter, test_iter = BucketIterator.splits(
    datasets=(train_dataset, test_dataset), 
    batch_sizes=(1000, 1000),
    sort=False,
    device = device
)

In [ ]:
class ModelParam(object):
    def __init__(self, param_dict: dict = dict()):
        self.input_size = param_dict.get('input_size', 0)
        self.vocab_size = param_dict.get('vocab_size')
        self.embedding_dim = param_dict.get('embedding_dim', 300)
        self.target_dim = param_dict.get('target_dim', 2)
        
class MyModel(nn.Module):
    def __init__(self, model_param: ModelParam):
        super().__init__()
        self.embedding = nn.Embedding(
            model_param.vocab_size, 
            model_param.embedding_dim
        )
        self.lin = nn.Linear(
            model_param.input_size * model_param.embedding_dim, 
            model_param.target_dim
        )
        
    def forward(self, x):
        features = self.embedding(x).view(x.size()[0], -1)
        features = F.relu(features)
        features = self.lin(features)
        return features

In [ ]:
def accuracy(outputs, labels):
    _, preds = torch.max(outputs, dim=1)
    return torch.tensor(torch.sum(preds == labels).item() / len(preds))

In [ ]:
model_param = ModelParam(
    param_dict=dict(
        vocab_size=len(text_field.vocab),
        input_size=5
    )
)

model = MyModel(model_param).to(device)
loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)
epochs = 1000

all_train_losses = list()
all_test_losses = list()
for epoch in range(epochs):
    epoch_losses = list()
    epoch_accuracies = list()
    for batch in train_iter:
        optimizer.zero_grad()
        prediction = model(batch.text.T)
        loss = loss_function(prediction, batch.label)
        loss.backward()
        optimizer.step()
        
        epoch_accuracies.append(accuracy(prediction, batch.label))
        epoch_losses.append(loss.item())
    all_train_losses.append(sum(epoch_accuracies)/len(epoch_accuracies))
    #print('train accuracy on epoch {} : {:.3f}'.format(epoch, sum(epoch_accuracies)/len(epoch_accuracies)))
    #print('train loss on epoch {} : {:.3f}'.format(epoch, np.mean(epoch_losses)))
    
    test_losses = list()
    test_accuracies = list()
    for batch in test_iter:
        with torch.no_grad():
            optimizer.zero_grad()
            prediction = model(batch.text.T)
            loss = loss_function(prediction, batch.label)
            
            test_accuracies.append(accuracy(prediction, batch.label))
            test_losses.append(loss.item())
    all_test_losses.append(sum(test_accuracies)/len(test_accuracies))
    #print('test accuracy on epoch {} : {:.3f}'.format(epoch, sum(test_accuracies)/len(test_accuracies)))
    
    #print('test loss on epoch {}: {:.3f}'.format(epoch, np.mean(test_losses)))
    if (epoch%100 == 0):
      print("Done with epoch ", epoch, ":")
print("Done!")

In [ ]:
epochs_list = [*range(0,epochs)]
plt.plot(epochs_list, all_train_losses)
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.title("Train Losses over 1000 epochs")
plt.show()

plt.plot(epochs_list, all_test_losses)
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.title("Train Losses over 1000 epochs")
plt.show()